<a href="https://colab.research.google.com/github/lucadonnoh/progetto-deblur/blob/main/Progetto_Deblur_CN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Progetto Deblur**

---

Un'azienda vuole commercializzare un dispositivo di acquisizione immagini, del quale si sa che acquisisce con rumore Gaussiano additivo e sfocatura Gaussiana.
Il modello è quindi:

\begin{align*}
    b = A x_{true} + \eta
\end{align*}

Dove $A \in \mathbb{R}^{mn}$ è la matrice di sfocamento, $\eta \sim \mathcal{N}(0, \sigma^2)$ rumore additivo con varianza $\sigma^2$ incognita, $b \in \mathbb{R}^{mn}$ è la vettorizzazione dell'immagine corrotta (acquisita) $B \in \mathbb{R}^{m \times n}$, $x_{true} \in \mathbb{R}^{mn}$ è la vettorizzazione dell'immagine originale $X_{true} \in \mathbb{R}^{m \times n}$.


1. Dopo aver importato le librerie necessarie, assegnare ad una variabile $X$ l'immagine $\texttt{cameraman}$, ottenibile tramite la libreria $\texttt{skimage.data}$ con il nome di $\texttt{camera()}$, effettuarne il $\texttt{reshape}$ e assegnarlo alla variabile $x$.

2. Tramite la funzione $\texttt{skimage.filters.gaussian}$ aggiungere sfocatura Gaussiana ad $X$ per ottenere $X_{blur}$, che rappresenta il reshape ad immagine di $x_{blur} = A x_{true}$, con $A$ matrice di sfocatura.

3. Caricare su una variabile $\eta$, della stessa dimensione di $X_{true}$, una realizzazione di rumore gaussiano con varianza $\sigma = 0.1$ (fare riferimento alle slides per la formula). 
Utilizzare $\eta$ per corrompere l'immagine $X_{blur}$ e calcolare $B = X_{blur} + \eta$. \\
Visualizzare i risultati ottenuti.

In [86]:
#Importo le altre foto
from google.colab import files
uploaded = files.upload()

Saving QRCode.png to QRCode (2).png
Saving TextImage.png to TextImage (2).png
Saving unknown.png to unknown (1).png


In [84]:
def gkern(kernlen, nsig):
  """
  Genera un kernel per la realizzazione di un filtro di sfocatura Gaussiana.

  Input ->
  kernlen: diametro della sfocatura gaussiana.
  nsig:    varianza della sfocatura. Più basso, più sfoca.

  Output -> Kernel Gaussiano di dimensione (kernlen+1) x (kernlen+1) e varianza nsig.
  """
  import scipy.stats as st

  x = np.linspace(-nsig, nsig, kernlen+1)
  kern1d = np.diff(st.norm.cdf(x))
  kern2d = np.outer(kern1d, kern1d)
  return kern2d/kern2d.sum()

def A(x, d=7, sigma=0.5):
  """
  Esegue il prodotto Ax, dove A è la matrice di sfocamento (che, per aumentare l'efficienza, non viene memorizzata).
  
  Input ->
  x:     Immagine di dimensione m x n, che si vuole sfocare.
  d:     Diametro della sfocatura Gaussiana.
  sigma: Varianza della sfocatura Gaussiana.
  
  Output -> Immagine di dimensione m x n, sfocata.
  """
  from scipy.signal import convolve2d
  from numpy import fft
  m, n = x.shape

  K_ext = np.zeros((m, n)) 
  K_ext[:d, :d] = gkern(d, sigma)
  K_ext = fft.fft2(K_ext)
  x = fft.fft2(x)

  return np.real(fft.ifft2(K_ext * x))

def AT(x, d=7, sigma=0.5):
  """
  Esegue il prodotto A^T x, dove A è la matrice di sfocamento (che, per aumentare l'efficienza, non viene memorizzata).
  
  Input ->
  x:     Immagine di dimensione m x n, che si vuole sfocare.
  d:     Diametro della sfocatura Gaussiana.
  sigma: Varianza della sfocatura Gaussiana.
  
  Output -> Immagine di dimensione m x n, sfocata.
  """
  from scipy.signal import convolve2d
  from numpy import fft
  m, n = x.shape

  K_ext = np.zeros((m, n))
  K_ext[:d, :d] = gkern(d, sigma)
  K_ext = fft.fft2(K_ext)
  x = fft.fft2(x)

  return np.real(fft.ifft2(np.conj(K_ext) * x))

In [87]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, filters
from skimage.io import imread

# https://imgur.com/a/kdBwySR
# https://imgur.com/a/B9L2XgB
# https://imgur.com/a/8LEt2IS

# 1
#X = data.camera()  Fotografo
#X = imread("QRCode.png")  #QRCode
X = imread("TextImage.png")  #Testo
X = imread("unknown.png") #Scimpanzè
m, n = X.shape
x = X.reshape(m*n) # array m*n


# 2
X_blur = A(X) # matrice
x_blur = X_blur.reshape(m*n) # array


# 3
# Generiamo il rumore eta
sigma = 0.1 # Varianza del rumore
eta = np.random.normal(size=X_blur.shape) # random gaussian distribution
eta /= np.linalg.norm(eta, 'fro') # norma di frobenius
eta *= sigma * np.linalg.norm(X_blur,'fro')

# Aggiungiamo il rumore all'immagine sfocata
B = X_blur + eta
b = B.reshape(m*n)

# Visualizziamo i risultati
plt.figure(figsize=(30, 10))

ax1 = plt.subplot(1, 3, 1)
ax1.imshow(X, cmap='gray')
plt.title('Immagine Originale', fontsize=30)

ax2 = plt.subplot(1, 3, 2)
ax2.imshow(X_blur, cmap='gray')
plt.title('Immagine Sfocata', fontsize=30)

ax3 = plt.subplot(1, 3, 3)
ax3.imshow(B, cmap='gray')
plt.title('Immagine Corrotta', fontsize=30)

plt.show()

ValueError: ignored

Il problema di ricostruire l'immagine originale partendo dalla sua corruzione $b$, si può riscrivere come un problema di ottimizzazione

\begin{align}
  x^* = \arg\min_x \frac{1}{2} ||Ax - b||_2^2 \hspace{2cm} (1)
\end{align}

4. Risolvere il problema (1) utilizzando l'algoritmo di discesa del gradiente con scelta del passo tramite backtracking, ricordando che, se 
\begin{align*}
  f(x) = \frac{1}{2} ||Ax - b||_2^2, \hspace{10px} \nabla f(x) = A^T(Ax - b)
\end{align*}
E chiamare la soluzione ottenuta $x_{naive}$.

5. Modificare l'algoritmo precedente a far restituire l'errore tra l'immagine ricostruita al passo $k$ e l'immagine originale, per ogni $k$. Visualizzare il grafico dell'errore, e individuare per quale valore di $k$ si ha semiconvergenza. Per tale valore, calcolare l'immagine ricostruita e chiamarla $x_{trunc}$.

In [ ]:
def plot( iters, error, title): 
  err_iteration = np.linspace(1, iters, error.size)
  plt.plot(err_iteration, error)
  plt.title(title)
  plt.grid()
  plt.show()

def find_x(error):
  return np.where(error == np.amin(error))[0]

def lambottimale(x, b, max):
  lamb=0.00000001
  k=0
  x,_,_ = minimize_reg(B,X,B,lamb, max, ABSOLUTE_STOP) #bisogna ogni volta cambiare la funzione minimize in base a che lambda buoi calcolare
  rat = 1.1
  while np.linalg.norm(A(x)-b)**2 <= np.linalg.norm(eta)**2:
    lamb *= rat
    x,_,_ = minimize_reg(B,X,B,lamb, max, ABSOLUTE_STOP)
    k+=1
    print(",", end='')
  print("it: ", k)
  return lamb/rat

MAXITERATIONS=50
ABSOLUTE_STOP=1.e-5





In [ ]:
# 4 - 5 
#Modificare l'algoritmo è stato interpretato come "sovrascrivere il precedente"

def f(x, b): #
  return 0.5*(np.linalg.norm(A(x)-b))**2

def grad_f(x, b):
  return AT(A(x)-b)

def next_step(x, b, f, grad):

  alpha = 1.1
  rho = 0.5
  c1 = 0.25
  k = 0

  while f(x-alpha*grad, b) > f(x, b) - alpha * c1 * np.linalg.norm(grad, 'fro') ** 2:
    alpha *= rho
    k += 1
  
  return alpha;

def minimize(x0, x_true, b, maxit, abstop):
  x = x0
  grad = grad_f(x, b)
  err = np.zeros(maxit) #errore alla k-esima iterazione

  k = 0
  while (np.linalg.norm(grad) > abstop) and (k < maxit):
    x = x + next_step(x, b, f, grad) * (-grad) # x + passo*direzione (-gradiente ovvero massima decrescita)
    grad = grad_f(x, b)
    err[k]  = np.linalg.norm(x_true - x)  #distanza con la norma2, prima fa la differenza membro a membro fra matrici poi ne calcola la norma2
    k += 1
    print(".", end='')

  err = err[0:k]  #l'ultima iterazione è esclusa perchè l'estremo destro viene escluso
  return(x, k, err)


In [ ]:
x_naive, ite_naive, err_naive = minimize(B, X, B, MAXITERATIONS, ABSOLUTE_STOP) #anche se nelle funzioni viene utilizzato 'x' viene passata 'X' ovvero la matrice dell'immagine
plot(ite_naive, err_naive,"Errore del metodo naive")

X_trunc_index = find_x(err_naive)
X_trunc,_,_= minimize(B,X,B,X_trunc_index+1, ABSOLUTE_STOP)
print(err_naive[X_trunc_index])

Per risolvere la semiconvergenza, si introduce il problema regolarizzato

\begin{align*}
  x^* = \arg\min_x \frac{1}{2} ||Ax - b||_2^2 + \frac{\lambda}{2} ||x||_2^2 \hspace{2cm} (2)
\end{align*}

con $\lambda > 0$ parametro di regolarizzazione.

6. Sfruttando il fatto che, se
\begin{align*}
f(x) = \frac{1}{2} ||Ax - b||_2^2 + \frac{\lambda}{2} ||x||_2^2, \hspace{10px} \nabla f(x) = A^T(Ax - b) + \lambda x
\end{align*}
Risolvere il problema (2) utilizzando l'algoritmo precedente per diversi valori di $\lambda$, e stimare il valore ottimale di $\lambda$ che minimizzi l'errore tra l'immagine ricostruita e l'immagine originale (metodo euristico per il parametro di regolarizzazione). Chiamare $x_{\lambda}$ tale soluzione. \\

7. Stimare il valore ottimale di $\lambda$ tramite principio di discrepanza, ovvero scegliere il più grande $\lambda$ tale che
\begin{align*}
  ||A x_\lambda - b||_2^2 \leq ||\eta||_2^2
\end{align*}
Dove $\eta$ è il rumore. Chiamare $x_\lambda^{disc}$ la soluzione ottenuta dal $\lambda$ calcolato con principio di discrepanza.
 

In [ ]:
def f_reg(x, b, lamb):
  return 0.5*(np.linalg.norm(A(x)-b))**2 + 0.5*lamb*np.linalg.norm(x)**2

def grad_f_reg(x, b, lamb):
  return AT(A(x)-b) + lamb*x

def next_step_reg(x, b, f, grad, lamb):

  alpha = 1.1
  rho = 0.5
  c1 = 0.25
  k = 0

  while f_reg(x-alpha*grad, b, lamb) > f_reg(x, b, lamb) - alpha * c1 * np.linalg.norm(grad, 'fro') ** 2:
    alpha *= rho
    k += 1
  
  return alpha;

def minimize_reg(x0, x_true, b, lamb, maxit, abstop):
  x = x0
  grad = grad_f_reg(x, b, lamb)
  err = np.zeros(maxit) #errore alla k-esima iterazione

  k = 0
  while (np.linalg.norm(grad) > abstop) and (k < maxit):
    x = x + next_step_reg(x, b, f, grad, lamb) * (-grad) # x + passo*direzione (-gradiente ovvero massima decrescita)
    grad = grad_f_reg(x, b, lamb)
    err[k]  = np.linalg.norm(x_true - x)  #distanza con la norma2, prima fa la differenza membro a membro fra matrici poi ne calcola la norma2
                                          #TODO: in teoria si potrebbe trasformare entrambi le matrici in vettori e poi farne la distanza ma ha senso? 
    k += 1

  err = err[:k]  #l'ultima iterazione è esclusa perchè l'estremo destro viene escluso
  return(x, k, err)


In [ ]:
def lambottimale(b, max):
  lamb= 0.6
  k=0
  x,_,err = minimize_reg(B,X,B,lamb, max, ABSOLUTE_STOP) #bisogna ogni volta cambiare la funzione minimize in base a che lambda buoi calcolare
  x_index=find_x(err)
  x,_,_ = minimize_reg(B, X, B, lamb, x_index+1, ABSOLUTE_STOP)
  rat = 0.9
  while np.linalg.norm(A(x)-b)**2 >= np.linalg.norm(eta)**2:
    lamb *= rat
    x,_,_ = minimize_reg(B,X,B,lamb, max, ABSOLUTE_STOP)
    x_index=find_x(err)
    x,_,_ = minimize_reg(B, X, B, lamb, x_index+1, ABSOLUTE_STOP)
    k+=1
    print(lamb)
    print(",", end='')
  print("it: ", k-1)
  return lamb/rat

lamb_reg = 0.03846986256908595 #lambda in modo euristico
#lamb_reg = lambottimale(B, MAXITERATIONS)
print(lamb_reg)
print(np.linalg.norm(A(X)-B)**2 <= np.linalg.norm(eta)**2)

(x_reg, ite_reg, err_reg) = minimize_reg(B, X, B, lamb_reg, MAXITERATIONS, ABSOLUTE_STOP) #anche se nelle funzioni viene utilizzato 'x' viene passata 'X' ovvero la matrice dell'immagine
plot(ite_reg, err_reg,"Errore del metodo ricostruito")

X_reg_index_reg = find_x(err_reg)
print(err_reg[X_reg_index_reg])
X_lambda,_,_= minimize_reg(B, X, B, lamb_reg, X_reg_index_reg+1, ABSOLUTE_STOP)


In [ ]:
lamb_disc = 0.06 #lamda tramite condizione discrepanza

(X_reg2, ite_reg2, err_reg2) = minimize_reg(B, X, B, lamb_disc, MAXITERATIONS, ABSOLUTE_STOP) #anche se nelle funzioni viene utilizzato 'x' viene passata 'X' ovvero la matrice dell'immagine
plot(ite_reg2, err_reg2,"Errore del metodo ricostruito")

X_reg_index2 = find_x(err_reg2)
print(err_reg2[X_reg_index2])
X_lambda_disc,_,_= minimize_reg(B, X, B, lamb_disc, X_reg_index2+1, ABSOLUTE_STOP)

8. Ripetere quanto fatto nel punto 6, utilizzando la norma 1 come regolarizzatore, ovvero:
\begin{align*}
    x^* = \arg\min_x \frac{1}{2} ||Ax - b||_2^2 + \lambda ||x||_1 \hspace{2cm} (3)
\end{align*}
Dove la funzione obiettivo ha gradiente
\begin{align*}
  A^T(Ax - b) + \lambda \hspace{2px} sign(x)
\end{align*}

In [ ]:
def f_reg_1(x, b, lamb): 
  return 0.5*(np.linalg.norm(A(x)-b))**2 + lamb*np.linalg.norm(x, 1)

def grad_f_reg_1(x, b, lamb):
  return AT(A(x)-b) + lamb*np.sign(x)

def next_step_reg_1(x, b, f, grad, lamb):

  alpha = 1.1
  rho = 0.5
  c1 = 0.25
  k = 0

  while f_reg_1(x-alpha*grad, b, lamb) > f_reg_1(x, b, lamb) - alpha * c1 * np.linalg.norm(grad, 'fro') ** 2:
    alpha *= rho
    k += 1
  
  return alpha;

def minimize_reg_1(x0, x_true, b, lamb, maxit, abstop):
  x = x0
  grad = grad_f_reg_1(x, b, lamb)
  err = np.zeros(maxit) #errore alla k-esima iterazione

  k = 0
  while (np.linalg.norm(grad) > abstop) and (k < maxit):
    x = x + next_step_reg_1(x, b, f, grad, lamb) * (-grad) # x + passo*direzione (-gradiente ovvero massima decrescita)
    grad = grad_f_reg_1(x, b, lamb)
    err[k]  = np.linalg.norm(x_true - x)  #distanza con la norma2, prima fa la differenza membro a membro fra matrici poi ne calcola la norma2
                                          #TODO: in teoria si potrebbe trasformare entrambi le matrici in vettori e poi farne la distanza ma ha senso? 
    k += 1
    print(".", end='')

  err = err[:k]  #l'ultima iterazione è esclusa perchè l'estremo destro viene escluso
  return(x, k, err)

In [ ]:
#Punto 8,  norma 1

lamb_1 = 4.392300000000001 #lamda tramite condizione discrepanza

(x_rec_1, ite_rec_1, err_reg_1) = minimize_reg_1(B, X, B, lamb_1, MAXITERATIONS, ABSOLUTE_STOP) #anche se nelle funzioni viene utilizzato 'x' viene passata 'X' ovvero la matrice dell'immagine
plot(ite_rec_1, err_reg_1,"Errore del metodo ricostruito con norma 1")

X_reg_index_1 = find_x(err_reg_1)
print(err_reg_1[X_reg_index_1])
X_lambda_1,_,_= minimize_reg_1(B, X, B, lamb_1, X_reg_index_1+1, ABSOLUTE_STOP)

9. ($\textit{facoltativo}$) Ripetere quanto fatto nei punti 6 e 8, utilizzando una norma mista come regolarizzatore, ovvero:
\begin{align*}
    x^* = \arg\min_x \frac{1}{2} ||Ax - b||_2^2 + \frac{\lambda}{2} ||x||_2^2 + \mu ||x||_1 \hspace{2cm} (4)
\end{align*}

In [ ]:
def f_reg_mista(x, b, lamb, mu): 
  return 0.5*(np.linalg.norm(A(x)-b))**2 + 0.5*lamb*np.linalg.norm(x)**2 + mu*np.linalg.norm(x,1)

def grad_f_reg_mista(x, b, lamb, mu):
  return AT(A(x)-b) + lamb*x +  mu*np.sign(x)

def next_step_reg_mista(x, b, f, grad, lamb, mu):

  alpha = 1.1
  rho = 0.5
  c1 = 0.25
  k = 0

  while f_reg_mista(x-alpha*grad, b, lamb, mu) > f_reg_mista(x, b, lamb, mu) - alpha * c1 * np.linalg.norm(grad, 'fro') ** 2:
    alpha *= rho
    k += 1
  
  return alpha;

def minimize_reg_mista(x0, x_true, b, lamb, mu, maxit, abstop):
  x = x0
  grad = grad_f_reg_mista(x, b, lamb, mu)
  err = np.zeros(maxit) #errore alla k-esima iterazione

  k = 0
  while (np.linalg.norm(grad) > abstop) and (k < maxit):
    x = x + next_step_reg_mista(x, b, f, grad, lamb, mu) * (-grad) # x + passo*direzione (-gradiente ovvero massima decrescita)
    grad = grad_f_reg_mista(x, b, lamb, mu)
    err[k]  = np.linalg.norm(x_true - x)  #distanza con la norma2, prima fa la differenza membro a membro fra matrici poi ne calcola la norma2
                                          #TODO: in teoria si potrebbe trasformare entrambi le matrici in vettori e poi farne la distanza ma ha senso? 
    k += 1
    print(".", end='')

  err = err[:k]  #l'ultima iterazione è esclusa perchè l'estremo destro viene escluso
  return(x, k, err)

In [ ]:
#Punto 9,  norma mista

lamb = 0.02155582137126536 #lamda tramite condizione discrepanza
mu = 0.025625815225418787

(x_reg_mista, ite_reg_mista, err_reg_mista) = minimize_reg_mista(B, X, B, lamb, mu, MAXITERATIONS, ABSOLUTE_STOP) #anche se nelle funzioni viene utilizzato 'x' viene passata 'X' ovvero la matrice dell'immagine
plot(ite_reg_mista, err_reg_mista,"Errore del metodo ricostruito con norma mista")

X_reg_mista = find_x(err_reg_mista)
print(err_reg_mista[X_reg_mista])
X_lambda_mista,_,_= minimize_reg_mista(B, X, B, lamb, mu, X_reg_mista+1, ABSOLUTE_STOP)

---

# Analisi dei risultati

---

Per ognuna delle soluzioni trovate, calcolare:

*   Errore relativo rispetto alla soluzione esatta $x_{true}$.
*   PSNR (Peak Signal to Noise Ratio).

Visualizzare le ricostruzioni ottenute con i vari metodi, e confrontare, sia numericamente che attraverso dei grafici, PSNR ed errore relativo ottenuti dalle ricostruzioni. Includere nel confronto, il valore delle metriche per l'immagine corrotta $b$.


In [ ]:
def plot_all():
  
  # Visualizziamo i risultati
  plt.figure(figsize=(40, 40))

  ax1 = plt.subplot(4,2, 1)
  ax1.imshow(X, cmap='gray')
  plt.title('Immagine Originale', fontsize=30)

  ax2 = plt.subplot(4,2, 2)
  ax2.imshow(X_blur, cmap='gray')
  plt.title('Immagine Sfocata', fontsize=30)

  ax3 = plt.subplot(4,2, 3)
  ax3.imshow(B, cmap='gray')
  plt.title('Immagine Corrotta', fontsize=30)

  ax3 = plt.subplot(4,2, 4)
  ax3.imshow(X_trunc, cmap='gray')
  plt.title('Immagine Naive', fontsize=30)

  ax3 = plt.subplot(4,2, 5)
  ax3.imshow(X_lambda, cmap='gray')
  plt.title('Immagine Regolarizzata', fontsize=30)

  ax3 = plt.subplot(4,2, 6)
  ax3.imshow(X_lambda_disc, cmap='gray')
  plt.title('Immagine Regolarizzata2', fontsize=30)

  ax3 = plt.subplot(4,2, 7)
  ax3.imshow(X_lambda_1, cmap='gray')
  plt.title('Immagine Regolarizzata con norma 1', fontsize=30)

  ax3 = plt.subplot(4,2, 8)
  ax3.imshow(X_lambda_mista, cmap='gray')
  plt.title('Immagine Regolarizzata con norma mista', fontsize=30)

  plt.show()

In [ ]:
def er(x, x_true):
  return np.linalg.norm(x-x_true)/np.linalg.norm(x_true)

def print_errors():
  er_naive = er(X_trunc, X)
  er_reg = er(X_lambda, X)
  er_reg2 = er(X_lambda_disc, X)
  er_1 = er(X_lambda_1, X)
  er_mista = er(X_lambda_mista, X)
  er_b = er(B, X)

  print("Errore percentuale immagine corrotta:", er_b*100, "%")
  print("Errore percentuale immagine naive:", er_naive*100, "%")
  print("Errore percentuale immagine regolarizzata:", er_reg*100 ,"%")
  print("Errore percentuale immagine regolarizzata2:", er_reg2*100, "%")
  print("Errore percentuale immagine regolarizzata con norma uno", er_1*100 ,"%")
  print("Errore percentuale immagine regolarizzata con norma mista", er_mista*100 , "%")
  print()

M,N = X.shape

def PSNR(A):
  return 20*np.log10(A.max()/((1/(M*N))*np.linalg.norm(eta)))

def print_PSNR():

  print("PSNR immagine corrotta:", PSNR(B))
  print("PSNR immagine naive:", PSNR(X_trunc))
  print("PSNR immagine immagine regolarizzata:", PSNR(X_lambda))
  print("PSNR immagine immagine regolarizzata2:", PSNR(X_lambda_disc))
  print("PSNR immagine regolarizzata con norma uno:", PSNR(X_lambda_1))
  print("PSNR immagine regolarizzata con norma mista:", PSNR(X_lambda_mista))


print_errors()
print_PSNR()
plot_all()

Ripetere tutti i passaggi, variando il livello del rumore (controllato dalla variabile $\sigma$), i parametri del kernel di sfocamento Gaussiano, e l'immagine di test. \\
Per la scelta delle immagini di test, utilizzare immagini **in bianco e nero, a scelta**, prese da internet, che rispettino i seguenti criteri:


*   Almeno un'immagine "geometrica", ovvero con pochi dettagli e contorni netti e ben contrastati (es. QRCode, Codice a Barre, Scacchiera...).
*   Almeno un'immagine "fotografica", ovvero con molti dettagli e livelli di grigio sfocati.
*   Almeno un'immagine contenente del testo, che diventerà difficilmente leggibile dopo il processo di sfocamento.


Osservare quale metodo di ricostruzione si comporta meglio nei vari esperimenti, ricordandosi corrompere l'immagine di input sia con rumore Gaussiano che con sfocatura.

